# ETL básico

### Introducción

La siguiente es una descripción básica para implementar flujos ETL (_Extract, Transform and Load_) en Python, utilizando la librería [`pygrametl`](https://chrthomsen.github.io/pygrametl/doc/index.html).

### Contexto

El siguiente caso consiste en un ejemplo simulado de un almacén de datos. 

<img src="./img/dm_ventas.vuerd.png" width="700px">

- El almacén de datos tiene una tabla de hechos y tres dimensiones organizadas en un __esquema estrella__. 
- La __tabla de hechos__ almacena datos de la cantidad de libros que se venden cada día. 
- La __dimensión Libro__ almacena el nombre y género de cada libro vendido.
- La __dimensión Localizacion__ almacena la comuna y región de los clientes.
- La __dimensión Tiempo__ almacena la fecha de cada venta. 

Para efectos de baja complejidad, ninguna de las dimensiones tiene forma de copo de nieve, tampoco contienen atributos que cambie lentamente. Sin embargo, la librería pygrametl es compatible con dichas características, si este fuera el caso ([más información](https://chrthomsen.github.io/pygrametl/doc/api/pygrametl.html)).

Para introducir los datos en el flujo ETL, se utilizan __dos fuentes de datos__: 
- La __base de datos__ (sistema OLTP) que contiene los registros de ventas
- Un __archivo CSV__ que contiene información de las comunas por regiones de Chile (obtenido desde [https://www.subdere.gov.cl/](https://www.subdere.gov.cl/documentacion/regiones-provincias-y-comunas-de-chile-2011)). Esta información es usada para complementar la dimensión Localización.

### Conexión con RDBMS

Se necesita crear conexiones a las dos bases de datos, que contiene los registros de ventas y el almacenén de datos, respectivamente. Estas conexiones deben ser conexiones [PEP 249](https://www.python.org/dev/peps/pep-0249/).


- Para establecer una conexión a RDBMS (_Relational Database Management System_), [`MySQL connector`](https://dev.mysql.com/doc/connector-python/en/) es una API que cumple con especificación Python Database API (PEP 249).


In [1]:
from mysql.connector import connect, Error, errorcode

def conexion(host, dbname, user, pwd):
    """
    Retorna un objeto conexión.
    """
    try:
        conn = connect(user=user, password=pwd, host=host, database=dbname)
        print('Conexión a {} establecida con exito!'.format(dbname))
        return conn
    except Error as e:
        if e.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print('Algo anda mal con tu usuario o contraseña.')
        elif e.errno == errorcode.ER_BAD_DB_ERROR:
            print('La base de datos no existe.')
        else:
            print(e)
    return None

### Conexión con archivos 

Para la extracción de filas de un archivo CSV no requiere una conexión PEP 249, solo un controlador de archivo abierto.
- pygrametl usa `DictReader` de Python para archivos CSV.
- Asume que el encabezado del archivo CSV contiene el nombre de cada columna. 
- Al usar `CSVSource`, es importante convertir los valores al tipo correcto antes de insertarlos en una tabla.

In [2]:
from pygrametl.datasources import CSVSource

region_file_handle = open('./data/cut_2010_v02.csv', 'r')
region_source = CSVSource(f=region_file_handle, delimiter=',')

### Extracción

__NOTA:__ La mayoría de las __abstracciones de la librería pigrametl producen, consumen y operan datos en forma de _registro___. Considere que un _registro_ es un diccionario cuyos nombres de las columnas son las claves, y los valores son los datos que contiene cada _registro_.

Recuperar datos a partir de consulta SQL.

In [18]:
from pygrametl.datasources import SQLSource

cnx_libreria = conexion('localhost', 'Libreria', 'root', 'mysqlroot')
query = """
    SELECT date(Pedido.Fecha) as 'fecha',
        Libro.Titulo AS 'libro',
        Genero.Nombre AS 'genero',
        Comuna.Nombre AS 'comuna',
        sum(Pedido.Cantidad) AS 'cantidad'
    FROM Pedido
    INNER JOIN Libro ON Libro.LibroID = Pedido.LibroID
    INNER JOIN Genero ON Genero.GeneroID = Libro.GeneroID
    INNER JOIN Cliente ON Cliente.ClienteID = Pedido.ClienteID
    INNER JOIN Comuna ON Comuna.ComunaID = Cliente.ComunaID
    GROUP BY fecha, libro, genero, comuna;
    """

name_mapping = 'Fecha', 'Titulo', 'Genero', 'Comuna', 'Cantidad'
ventas_source = SQLSource(connection=cnx_libreria, query=query, names=name_mapping)

Conexión a Libreria establecida con exito!


### Transformación

A partir de la fecha obtenida desde la base de datos __Libreria__ se obtienen nuevos atributos para poblar la dimensión Tiempo. Para esto, se diseñan dos funciones, una para divir la fehca en __dia, mes y año__, y la otra, para obtener el trimestre al que corresponde dicha fecha.

In [4]:
def set_dmY(row):
    """Agrega los componente de la fecha al registro (row)

    Args:
        row (dict): Registros que se carga en el almacen
    """

    fecha = row['Fecha']
    row['Dia'] = fecha.day
    row['Mes'] = fecha.month
    row['Anio'] = fecha.year

In [5]:
def set_trimestre(row):
    """Agrega el trimestre al registro (row) 

    Args:
        row (dict): Registros que se carga en el almacen
    """
    
    if row['Mes'] < 4:
        trimestre = 1
    elif row['Mes'] < 7:
        trimestre = 2
    elif row['Mes'] < 10:
        trimestre = 3
    else:
        trimestre = 4
        
    row['Trimestre'] = trimestre

### Conexion con el almacén de datos

- `ConnectionWrapper` comparte automáticamente entre abstracciones de pygrametl, se guarda en una variable para que la conexión se pueda cerrar.

In [6]:
from pygrametl import ConnectionWrapper

cnx_dm = conexion('localhost', 'VentasDM', 'root', 'mysqlroot')
cnx_dm_wrapper = ConnectionWrapper(connection=cnx_dm)

Conexión a VentasDM establecida con exito!


### Carga

- pygrametl proporciona diversos tipos de abstracciones para dimensiones y tablas de hechos. En este ejemplo, se usan las más simples.
- `CachedDimension` crea una instancia de para cada dimensión en el almacén de datos. 
- `CachedDimension` usa un caché local para reducir significativamente la cantidad de solicitudes emitidas al RDBMS. 

Para cada dimensión, __se proporciona el nombre de la tabla de la base de datos, la clave principal de la tabla y las columnas sin clave (atributos) de la tabla__. 

Además, para la __dimensión de Localicalizacion__, se proporciona el subconjunto de los atributos que se deben usar para buscar la clave principal.

In [7]:
from pygrametl.tables import CachedDimension, FactTable

libro_dim = CachedDimension(
        name='Libro',
        key='LibroID',
        attributes=['Titulo', 'Genero'])

tiempo_dim = CachedDimension(
        name='Tiempo',
        key='TiempoID',
        attributes=['Fecha', 'Dia', 'Mes', 'Anio', 'Trimestre'])

localizacion_dim = CachedDimension(
        name='Localizacion',
        key='LocalizacionID',
        attributes=['Comuna', 'Region'],
        lookupatts=['Comuna'])

A continuación se crea una instancia de `FactTable` para la tabla de hechos del almacén de datos, a partir, del nombre de la tabla, una lista de columnas que constituyen la clave principal de la tabla de hechos y una lista de las medidas.

In [8]:
ventas_ft = FactTable(
        name='Ventas',
        keyrefs=['LibroID', 'LocalizacionID', 'TiempoID'],
        measures=['Cantidad'])

La __dimensión Localizacion__ se completa con datos del archivo CSV, ya que el archivo contiene toda la información necesaria para ambas columnas de la tabla. Para insertar las filas se usa el método `CachedDimension.insert()`

__NOTA__: Si la dimensión Localizacion se completara solo con los datos de la base de datos Libreria, sería necesario actualizar el atributo de región con datos del archivo CSV cada vez que se actualicen los datos del almacen de datos.

In [9]:
for row in region_source:
    localizacion_dim.insert({'Comuna': row['Nombre Comuna'], 'Region': row['Nombre Región']})

region_file_handle.close()

A continuación se cargan los datos en las __dimensiones Libro, Tiempo, y también en la tabla de hechos__.

In [10]:
for row in ventas_source:
    # Se divide la fecha
    set_dmY(row)
    
    # Se obtiene el trimestre
    set_trimestre(row)

    # La fila se actualiza con las claves primarias correctas para 
    # cada dimensión y cualquier dato nuevo se inserta en cada una 
    # de las dimensiones al mismo tiempo.
    row['LibroID'] = libro_dim.ensure(row)
    row['TiempoID'] = tiempo_dim.ensure(row)

    # CachedDimension.ensure() no se utiliza para la dimensión Localizacion 
    # porque ya se ha rellenado. En su lugar, se utiliza el método 
    # CachedDimension.lookup() que no inserta ningún dato y devuelve el 
    # valor None si no está disponible una fila con las búsquedas correctas. 
    # En este caso, se genera un error si falta una ubicación en el archivo 
    # CSV, ya que la recuperación no es posible.
    row['LocalizacionID'] = localizacion_dim.lookup(row)
    if not row['LocalizacionID']:
        raise ValueError("La comuna no se encuantra en la dimension Localizacion")

    # Considerando que la cantidad de ordenes se presenta como un valor agregado 
    # en los registros de ventas, la fila se puede insertar en el almacén de datos. 
    # De lo contrario, se debería realizar la transformacion antes de insertar.
    ventas_ft.insert(row)

# Después de que se hayan insertado todos los datos, se ordena a la conexión que se 
# confirme la insersión y luego se cierra. Esto garantiza que los datos se
# confirmen en la base de datos y que los recursos utilizados por la conexión se
# liberen.
cnx_dm_wrapper.commit()
cnx_dm_wrapper.close()

# Finalmente, se cierra la conexión a la base de datos Libreria
cnx_libreria.close()